### Importando Bibliotecas e Configurando o Ambiente

In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
import re

import unicodedata

In [ ]:
data = pd.read_excel('base_fic.xlsx')
data.head()

ANALISE_RESPOSTA  \
0                  Resposta Completa   
1                  Resposta Completa   
2  Devolvido - Resposta inconclusiva   
3       Devolvido - Falta de contato   
4  Devolvido - Resposta inconclusiva   

                         Sacl Cod Texto Manifestacao  \
0  O cliente solicita informações detalhadas sobr...   
1  Solicitação de atualização no contrato de serv...   
2  O cliente deseja cancelar o pedido realizado r...   
3  Reclamação sobre o atendimento recebido na últ...   
4  O cliente deseja cancelar o pedido realizado r...   

                                   Sacl Texto Tarefa  
0  Atendimento concluído com sucesso, todas as dú...  
1  Tentativa de contato realizada sem sucesso, nú...  
2  Encaminhado pedido de devolução para o setor r...  
3  Problema técnico solucionado conforme solicita...  
4  Atendimento concluído com sucesso, todas as dú...

### Pré-processamento do Texto

In [18]:
# import nltk
# nltk.download('stopwords')

In [19]:
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')

In [ ]:

def preprocess_text(text):
    text = text.lower()
    nfkd = unicodedata.normalize('NFKD', text)
    text = ''.join([c for c in nfkd if not unicodedata.combining(c)])
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

data['Sacl Cod Texto Manifestacao'] = data['Sacl Cod Texto Manifestacao'].apply(preprocess_text)
data['Sacl Texto Tarefa'] = data['Sacl Texto Tarefa'].apply(preprocess_text)
data.head()


ANALISE_RESPOSTA  \
0                  Resposta Completa   
1                  Resposta Completa   
2  Devolvido - Resposta inconclusiva   
3       Devolvido - Falta de contato   
4  Devolvido - Resposta inconclusiva   

                         Sacl Cod Texto Manifestacao  \
0  cliente solicita informacoes detalhadas sobre ...   
1           solicitacao atualizacao contrato servico   
2  cliente deseja cancelar pedido realizado recen...   
3  reclamacao sobre atendimento recebido ultima i...   
4  cliente deseja cancelar pedido realizado recen...   

                                   Sacl Texto Tarefa  
0  atendimento concluido sucesso todas duvidas cl...  
1  tentativa contato realizada sucesso numero inf...  
2     encaminhado pedido devolucao setor responsavel  
3  problema tecnico solucionado conforme solicita...  
4  atendimento concluido sucesso todas duvidas cl...

### Tokenização e Codificação das Classes

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

data['combined_text'] = data['Sacl Cod Texto Manifestacao'] + " " + data['Sacl Texto Tarefa']

# Tokenizando usando CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['combined_text']).toarray()

# Codificando as classes de saída
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['ANALISE_RESPOSTA'])


### Dividindo os Dados em Conjuntos de Treino e Teste

In [22]:

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Criando um Dataset Customizado com PyTorch

In [23]:

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Criando os datasets de treino e teste
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

# Criando DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


### Definindo o Modelo de Rede Neural

In [ ]:

class TextClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Camada de saída

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Inicializando o modelo
input_dim = X_train.shape[1]
hidden_dim = 128  # Número de neurônios na camada oculta
output_dim = len(label_encoder.classes_)
model = TextClassificationModel(input_dim, hidden_dim, output_dim)


### Configurando a Função de Perda e o Otimizador

In [25]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


### Treinando o Modelo

In [26]:

# Treinamento do modelo
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 1.1001316217275767
Epoch 2/10, Loss: 1.0775305399527917
Epoch 3/10, Loss: 1.0633968114852905
Epoch 4/10, Loss: 1.0529534541643584
Epoch 5/10, Loss: 1.0474792810586782
Epoch 6/10, Loss: 1.0389760457552397
Epoch 7/10, Loss: 1.0332094568472643
Epoch 8/10, Loss: 1.0239418469942534
Epoch 9/10, Loss: 1.0208973838732793
Epoch 10/10, Loss: 1.0130276863391583


### Avaliando o Modelo

In [27]:

# Avaliação do modelo nos dados de teste
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test data: {accuracy * 100:.2f}%")


Accuracy on test data: 30.00%
